In [ ]:
### REFERENCE CODE FROM RESOURCES SHARED ON THE SLACK
#
#
#

In [ ]:
## From 'SciPy documentation'

#Generate a test signal, a 2 Vrms sine wave whose frequency is slowly modulated around 3kHz, 
#corrupted by white noise of exponentially decreasing magnitude sampled at 10 kHz.
    
fs = 10e3
N = 1e5
amp = 2 * np.sqrt(2)
noise_power = 0.01 * fs / 2
time = np.arange(N) / float(fs)
mod = 500*np.cos(2*np.pi*0.25*time)
carrier = amp * np.sin(2*np.pi*3e3*time + mod)
noise = np.random.normal(scale=np.sqrt(noise_power), size=time.shape)
noise *= np.exp(-time/5)
x = carrier + noise
    
#Compute and plot the spectrogram.
    
f, t, Sxx = signal.spectrogram(x, fs)
plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()
#help(signal.spectrogram)

In [ ]:
## From 'intro to audio processing for python' article

#To read wav file
Fs, data = read('test.wav') #Fs = sample rate

Fs, data = read("Audiomoth1_Sakaroa_20220507_070000.WAV")

#To play audio
Audio(data, rate=Fs)

#To plot wave
#plt.figure()
#plt.plot(data)
#plt.xlabel("Sample Index")
#plt.ylabel("Amplitude")
#plt.title("Waveform of Test Audio")
#plt.show()


In [46]:
## From BirdNet github page

#def openAudioFile(path, sample_rate=48000, offset=0.0, duration=None):
    
    # Open file with librosa (uses ffmpeg or libav)
    #sig, rate = librosa.load(path, sr=sample_rate, offset=offset, duration=duration, mono=True, res_type='kaiser_fast')

    #return sig, rate

def openAudioFile(path, sample_rate=48000, offset=0.0, duration=None):
    
    # Open file with librosa (uses ffmpeg or libav)
    sig, rate = librosa.load("Audiomoth1_Sakaroa_20220507_070000.WAV", sr=sample_rate, offset=offset, duration=duration, mono=True, res_type='kaiser_fast')

    return sig, rate
    
#sig, rate = librosa.load("Audiomoth1_Sakaroa_20220507_070000.WAV", sr=48000, offset=0.0, duration=None, mono=True, res_type='kaiser_fast')
#print(sig, rate) 
#sig, rate


In [47]:
## From BirdNet github page

def specsFromFile(path, rate, offset=0.0, duration=None, **kwargs):

    # Open file
    sig, rate = openAudioFile(path, rate, offset, duration)

    # Yield all specs for file
    for spec in specsFromSignal(sig, rate, **kwargs):
        yield spec

In [ ]:
## From BirdNet github page

RANDOM = np.random.RandomState(1337)
CACHE = {}

def buildBandpassFilter(rate, fmin, fmax, order=4):

    global CACHE

    fname = 'bandpass_' + str(rate) + '_' + str(fmin) + '_' + str(fmax)
    if not fname in CACHE:
        wn = np.array([fmin, fmax]) / (rate / 2.0)
        filter_sos = scipy.signal.butter(order, wn, btype='bandpass', output='sos')

        # Save to cache
        CACHE[fname] = filter_sos

    return CACHE[fname]

In [ ]:
## From BirdNet github page

RANDOM = np.random.RandomState(1337)
CACHE = {}

def applyBandpassFilter(sig, rate, fmin, fmax):

    # Build filter or load from cache
    filter_sos = buildBandpassFilter(rate, fmin, fmax)

    return scipy.signal.sosfiltfilt(filter_sos, sig)

In [48]:
## From BirdNet github page

RANDOM = np.random.RandomState(1337)
CACHE = {}

def spectrogram(sig, rate, shape=(64, 512), win_len=512, fmin=150, fmax=15000, frequency_scale='mel', magnitude_scale='nonlinear', bandpass=True, decompose=False):

    # Compute overlap
    hop_len = int(len(sig) / (shape[1] - 1)) 
    win_overlap = win_len - hop_len + 2
    #print 'WIN_LEN:', win_len, 'HOP_LEN:', hop_len, 'OVERLAP:', win_overlap

    # Adjust N_FFT?
    if frequency_scale == 'mel':
        n_fft = win_len
    else:
        n_fft = shape[1] * 2

    # Bandpass filter?
    if bandpass:
        sig = applyBandpassFilter(sig, rate, fmin, fmax)

    # Compute spectrogram
    f, t, spec = scipy.signal.spectrogram(sig,
                                          fs=rate,
                                          window=scipy.signal.windows.hann(win_len),
                                          nperseg=win_len,
                                          noverlap=win_overlap,
                                          nfft=n_fft,
                                          detrend=False,
                                          mode='magnitude')
    
    # Flip spectrum vertically (only for better visialization, low freq. at bottom)
    spec = spec[::-1, ...]
    